## IMPORT DES BIBLIOTHEQUES

In [1]:
from djitellopy import Tello
import tellopy
import cv2 as cv 
import socket, time
from time import sleep
import datetime
import os
from threading import Thread

# Utilisation  de `Tellopy`

* Décollage et attérissage 

In [2]:

# Paramètres de communication
host = ''
port = 9000
locaddr = (host,port)

#Connexion à l'adresse IP du drône pour entamer la communication 
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
tello_address = ('192.168.10.1', 8889)
sock.bind(locaddr)


sock.sendto('command'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Takeoff
sock.sendto('takeoff'.encode(encoding="utf-8"), tello_address)
time.sleep(5)


#Rotate clockwise 360
sock.sendto('tcw 360'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Land
sock.sendto('land'.encode(encoding="utf-8"), tello_address)
time.sleep(5)

# Utilisation de  `Djitellopy`

*  Décollage , photo et attérissage 

In [1]:
from djitellopy import Tello
tello= Tello () 
tello.connect() # Connexion
'''
tello.takeoff() # Décollage 
tello.streamon() # Activation de la caméra

time.sleep(2) # Tempo de 2s 
frame_read = tello.get_frame_read() # Capture d'impage 
cv.imwrite("testnew.png", frame_read.frame) # Stockage en local 
time.sleep(2)
tello.land()   # Attérissage
'''
print("La  batterie  fait {}".format(tello.get_battery()))

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'


La  batterie  fait 58


*  Capture de vidéo

In [2]:
from threading import Thread

tello = Tello()
tello.connect()

KeepRecording= True
tello.streamon()

frame_read = tello.get_frame_read()

def videoRecorder():
    height, width,_= frame_read.frame.shape
    video = cv.VideoWriter('Video.avi', cv.VideoWriter_fourcc(*'XVID'), 30, (width,height))
    
    while KeepRecording:
        video.write(frame_read.frame)
        time.sleep(1/30)
    video.release()
    
recorder  = Thread(target=videoRecorder)
recorder.start()

tello.takeoff()
tello.land()
KeepRecording= False
recorder.join()



[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


NameError: name 'Thread' is not defined

# HAND TRACKING

In [1]:

class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
 
        self.mpHands = mp.solutions.hands
        #self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        #self.detectionCon, self.trackCon)
        self.hands = self.mpHands.Hands()
        
        self.mpDraw = mp.solutions.drawing_utils
 
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
 
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handNo=0, draw=True):
 
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                print(id, cx, cy)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        return lmList
                    

In [2]:
import cv2
import time
import numpy as np
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from djitellopy import Tello
import mediapipe as mp

### PARAMETRES DE LA CAMERA
width= 640
height= 480


########### CONNEXION A TELLO ET ACTIVATION DE LA CAMERA 
tello =  Tello()
tello.connect()
tello.streamon()

 
#### initialisation du temps de départ et de fin
pTime = 0
cTime = 0

# INSTANTIATION DE L'objet DETECTEUR DE MAINS
detector= handDetector(detectionCon= 0.7)

#ACCES AU VOLUME DE L'ORDINATEUR
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()

# Récupération du volume minimum et maximum
volRange = volume.GetVolumeRange() 
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

while True:  # BOUCLE INFINIE
    
    # CApture de l'image en temps réel
    frame_read = tello.get_frame_read()
    my_frame= frame_read.frame
    img = cv2.resize(my_frame,(width, height))
    
    # Detection de la main et des 21 features avec les positions en X et Y de chaque feature
    img= detector.findHands(img)
    lmList=  detector.findPosition(img, draw=  False)
  
    
    
    if len(lmList) != 0:
        # print(lmList[4], lmList[8])
        x1, y1 = lmList[4][1], lmList[4][2] # Coordonnées X et Y du feature 4
        x2, y2 = lmList[8][1], lmList[8][2] # Coordonnées X et  Y du feature 8 
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2 # Coordonnées du milieu entre 4 et 8 
        
        
        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        
        length = math.hypot(x2 - x1, y2 - y1) # Calcul de la distance entre les doigts
        
        # Corrélation entre le volume et la distance entre les doigts
        vol = np.interp(length, [50, 300], [minVol, maxVol])
        volBar = np.interp(length, [50, 300], [400, 150])
        volPer = np.interp(length, [50, 300], [0, 100])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None)
        
        # Passage de la couleur en vert lorsque la distannce est minime
        if length < 50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
 
    cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX,
                1, (255, 0, 0), 3)
 
    # Calcul de la fréquence de rafraichissement de l'image
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

        
    # Affichage de la fréquence de rafraichissement
    cv2.putText(img, str(int(fps)), (40, 50), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)

    cv2.imshow("Image", img)
    cv2.waitKey(1)

[INFO] tello.py - 122 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
[INFO] tello.py - 437 - Send command: 'command'
[INFO] tello.py - 461 - Response command: 'ok'
[INFO] tello.py - 437 - Send command: 'streamon'
[INFO] tello.py - 461 - Response streamon: 'ok'


0 264 211
1 310 183
2 344 122
3 364 65
4 375 15
5 295 28
6 307 -27
7 314 -65
8 317 -93
9 245 24
10 244 -37
11 237 -76
12 231 -104
13 202 40
14 185 -20
15 168 -56
16 157 -83
17 163 75
18 131 29
19 108 -2
20 91 -32
122 -46.304337981532555
0 274 225
1 318 191
2 347 121
3 364 58
4 377 6
5 301 30
6 312 -30
7 317 -70
8 319 -99
9 254 26
10 250 -38
11 244 -77
12 239 -105
13 210 40
14 192 -21
15 179 -58
16 170 -86
17 171 72
18 139 24
19 119 -6
20 105 -35
119 -46.99196478537817
0 276 237
1 319 198
2 348 129
3 365 64
4 377 8
5 302 37
6 316 -23
7 321 -64
8 325 -93
9 257 32
10 257 -34
11 253 -75
12 251 -105
13 213 44
14 199 -20
15 188 -62
16 182 -93
17 172 74
18 142 24
19 121 -11
20 108 -44
113 -48.65035404933138
0 272 243
1 317 205
2 349 136
3 366 69
4 374 12
5 299 45
6 315 -15
7 324 -55
8 331 -84
9 255 38
10 256 -29
11 255 -71
12 256 -102
13 212 50
14 199 -16
15 189 -59
16 185 -92
17 172 79
18 142 29
19 120 -7
20 104 -41
105 -50.84533072499325
0 270 247
1 316 210
2 347 140
3 365 73
4 374 14
5 295

11 486 220
12 503 232
13 501 94
14 489 189
15 503 221
16 515 236
17 536 96
18 523 176
19 527 207
20 530 223
59 -62.77982422779948
0 564 1
1 525 45
2 504 97
3 507 142
4 521 177
5 462 99
6 457 181
7 471 217
8 482 234
9 478 99
10 473 192
11 489 225
12 505 240
13 502 99
14 495 191
15 507 224
16 517 243
17 535 100
18 525 176
19 526 207
20 526 226
69 -60.273986297575384
0 563 10
1 531 58
2 513 110
3 517 155
4 531 190
5 462 112
6 461 192
7 478 225
8 491 240
9 475 111
10 472 202
11 492 230
12 510 240
13 498 110
14 495 202
15 511 229
16 523 242
17 531 109
18 523 185
19 528 213
20 532 227
64 -61.58784574030027
0 564 11
1 539 64
2 522 118
3 525 161
4 535 196
5 464 114
6 462 194
7 477 229
8 488 247
9 475 112
10 474 203
11 491 237
12 507 254
13 497 111
14 495 202
15 510 234
16 519 254
17 529 111
18 522 188
19 529 217
20 533 233
69 -60.19856331668671
0 566 10
1 537 61
2 519 115
3 522 159
4 533 194
5 464 113
6 461 192
7 477 228
8 490 244
9 476 112
10 473 201
11 492 234
12 509 251
13 499 111
14 495 20

6 282 -17
7 284 -48
8 282 -67
9 231 24
10 231 -22
11 227 -53
12 223 -77
13 192 37
14 182 -12
15 173 -43
16 164 -66
17 156 69
18 127 28
19 106 -1
20 90 -26
115 -48.05439025246804
0 276 259
1 318 205
2 341 125
3 354 55
4 360 -4
5 280 31
6 284 -41
7 283 -91
8 283 -129
9 236 29
10 226 -47
11 217 -96
12 211 -133
13 195 45
14 172 -27
15 157 -73
16 149 -110
17 157 79
18 120 23
19 103 -16
20 93 -53
146 -39.98185763897211
0 256 206
1 304 179
2 337 123
3 352 72
4 355 28
5 277 31
6 278 -23
7 276 -60
8 271 -88
9 231 28
10 229 -27
11 222 -65
12 217 -95
13 193 42
14 183 -14
15 171 -49
16 162 -76
17 158 73
18 128 31
19 106 0
20 89 -25
143 -40.91955120654649
0 272 261
1 314 208
2 338 128
3 350 56
4 356 -4
5 281 29
6 284 -43
7 281 -93
8 279 -130
9 236 26
10 225 -50
11 214 -98
12 207 -136
13 194 43
14 170 -29
15 154 -76
16 144 -114
17 155 78
18 117 22
19 99 -17
20 88 -54
147 -39.75939277852514
0 261 208
1 306 176
2 337 119
3 351 66
4 351 21
5 274 27
6 276 -25
7 275 -63
8 270 -89
9 228 25
10 226 -28
11 2

12 192 -126
13 183 52
14 156 -19
15 138 -65
16 126 -104
17 145 88
18 107 34
19 86 -5
20 72 -43
144 -40.598371573630935
0 258 260
1 300 208
2 328 133
3 340 66
4 331 9
5 268 36
6 271 -36
7 268 -83
8 264 -120
9 225 35
10 213 -41
11 200 -87
12 190 -125
13 185 52
14 157 -19
15 139 -64
16 127 -103
17 146 87
18 107 34
19 86 -5
20 72 -44
145 -40.36061769084794
0 261 260
1 302 209
2 329 132
3 340 64
4 331 6
5 268 37
6 270 -36
7 267 -82
8 263 -119
9 225 35
10 212 -39
11 199 -86
12 189 -123
13 185 53
14 157 -17
15 139 -62
16 126 -101
17 147 89
18 108 36
19 87 -3
20 73 -42
142 -41.15996595316585
0 260 257
1 301 205
2 329 130
3 341 63
4 333 7
5 267 36
6 270 -35
7 267 -82
8 264 -118
9 224 35
10 211 -40
11 199 -86
12 189 -124
13 184 52
14 156 -17
15 138 -63
16 126 -102
17 146 88
18 108 35
19 86 -3
20 71 -42
142 -41.03453735695745
0 259 255
1 300 204
2 329 129
3 340 62
4 333 5
5 268 36
6 271 -35
7 268 -81
8 266 -117
9 224 35
10 211 -39
11 199 -86
12 190 -123
13 184 52
14 156 -17
15 139 -62
16 127 -101

...

0 221 240
1 262 223
2 279 199
3 286 186
4 284 180
5 234 150
6 222 101
7 208 64
8 189 26
9 188 155
10 158 100
11 133 60
12 108 20
13 150 175
14 116 129
15 89 89
16 67 50
17 120 203
18 84 176
19 57 151
20 36 124
180 -31.073422090945442
0 261 319
1 307 271
2 328 219
3 338 171
4 350 127
5 238 160
6 218 98
7 206 60
8 194 23
9 197 167
10 159 97
11 134 49
12 112 8
13 167 189
14 123 130
15 95 88
16 73 52
17 144 222
18 98 182
19 68 155
20 43 128
187 -29.365458089402736
0 217 224
1 256 208
2 271 190
3 282 182
4 282 177
5 233 150
6 220 101
7 205 63
8 187 25
9 189 155
10 160 102
11 137 62
12 111 21
13 152 173
14 118 130
15 90 89
16 68 50
17 122 197
18 86 173
19 59 147
20 36 122
179 -31.516887566131295
0 261 321
1 307 273
2 329 220
3 339 171
4 350 127
5 238 160
6 219 99
7 207 60
8 194 24
9 197 168
10 158 98
11 133 52
12 112 11
13 166 190
14 123 131
15 96 89
16 74 52
17 143 223
18 97 184
19 68 156
20 43 129
186 -29.50975153783289
0 214 225
1 254 205
2 269 184
3 275 174
4 270 169
5 229 146
6 219 96
7

12 120 5
13 164 191
14 120 133
15 95 90
16 75 53
17 141 227
18 96 186
19 67 157
20 47 129
71 -59.737779820290896
0 273 325
1 315 271
2 329 219
3 322 175
4 304 134
5 235 157
6 212 96
7 224 90
8 243 103
9 195 168
10 158 97
11 138 49
12 120 4
13 165 191
14 121 132
15 95 90
16 75 53
17 141 227
18 96 185
19 68 156
20 48 129
68 -60.441038048083534
0 272 323
1 313 270
2 328 219
3 322 175
4 305 134
5 236 156
6 213 94
7 226 90
8 245 105
9 195 167
10 157 96
11 137 46
12 119 0
13 164 191
14 121 132
15 95 89
16 75 52
17 140 225
18 96 184
19 68 155
20 47 127
66 -60.90674380686583
0 272 323
1 312 269
2 327 219
3 322 174
4 305 133
5 235 157
6 213 95
7 226 90
8 245 105
9 195 168
10 157 97
11 137 48
12 119 3
13 165 191
14 121 131
15 94 89
16 74 52
17 141 226
18 96 184
19 67 155
20 47 127
66 -61.01872504703429
0 267 326
1 308 269
2 325 220
3 320 175
4 299 137
5 235 157
6 214 95
7 226 92
8 244 109
9 194 168
10 157 95
11 137 45
12 118 0
13 163 191
14 121 130
15 94 88
16 74 51
17 140 226
18 95 182
19 67 15

0 235 240
1 269 207
2 285 175
3 304 159
4 319 153
5 229 149
6 227 103
7 254 109
8 276 126
9 188 160
10 166 99
11 148 58
12 132 25
13 155 182
14 118 136
15 97 101
16 81 75
17 131 211
18 88 183
19 62 157
20 44 135
50 -65.04798362512331
0 266 325
1 306 266
2 320 216
3 313 171
4 287 139
5 232 159
6 214 99
7 230 100
8 250 119
9 193 169
10 158 101
11 141 57
12 123 16
13 162 191
14 121 131
15 95 90
16 74 52
17 138 225
18 95 183
19 66 155
20 46 128
42 -65.25
0 267 323
1 306 265
2 321 215
3 311 170
4 285 140
5 232 159
6 214 99
7 230 100
8 249 119
9 193 169
10 158 101
11 141 57
12 123 17
13 162 190
14 120 130
15 94 90
16 73 53
17 138 225
18 94 182
19 66 155
20 46 128
41 -65.25
0 258 326
1 301 267
2 318 217
3 311 170
4 286 139
5 231 159
6 214 99
7 230 100
8 250 120
9 192 168
10 158 99
11 141 56
12 122 15
13 160 188
14 119 128
15 93 88
16 73 51
17 135 223
18 92 180
19 64 153
20 45 127
40 -65.25
0 273 322
1 310 264
2 322 215
3 313 171
4 288 140
5 232 157
6 213 98
7 229 99
8 249 119
9 193 169
10 157

1 302 268
2 315 223
3 321 183
4 333 149
5 228 171
6 208 114
7 193 75
8 178 38
9 186 179
10 140 113
11 108 66
12 80 24
13 156 201
14 106 148
15 75 107
16 47 72
17 138 233
18 85 198
19 51 174
20 22 152
190 -28.54132371133653
0 262 311
1 300 268
2 314 223
3 322 183
4 338 149
5 227 170
6 207 113
7 193 73
8 178 36
9 185 179
10 140 113
11 108 66
12 79 23
13 155 201
14 105 147
15 73 106
16 45 72
17 136 233
18 84 198
19 51 174
20 23 152
195 -27.17530294466284
0 260 314
1 298 270
2 315 224
3 327 182
4 347 148
5 228 170
6 208 111
7 193 71
8 178 33
9 186 179
10 140 113
11 108 65
12 79 22
13 155 202
14 105 147
15 74 106
16 47 71
17 136 234
18 84 199
19 51 175
20 23 153
204 -24.947360833788174
0 260 312
1 300 269
2 317 222
3 327 181
4 344 146
5 229 169
6 208 110
7 194 70
8 179 32
9 186 177
10 139 109
11 107 61
12 78 18
13 156 200
14 106 146
15 75 105
16 48 70
17 138 232
18 86 197
19 52 174
20 22 154
200 -25.9559961313619
0 261 313
1 302 270
2 319 224
3 327 183
4 342 149
5 228 169
6 208 109
7 193 69

ZeroDivisionError: float division by zero